# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
airbnb = pd.read_csv('airbnb_hw.csv')
airbnb.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [21]:
# Display unique values in the 'Price' column before cleaning.
print("Unique values in 'Price' before cleaning:", airbnb['Price'].unique(),'\n')

# Clean the 'Price' variable by removing '$' and ',' and converting to numeric, coercing errors to NaN.
airbnb['Price'] = airbnb['Price'].astype(str).str.replace('[$,]', '', regex=True)
airbnb['Price'] = pd.to_numeric(airbnb['Price'], errors='coerce')

# Calculate the number of missing values after cleaning.
missing_prices_count = airbnb['Price'].isnull().sum()

print(f"Number of missing values in the cleaned 'Price' variable: {missing_prices_count}")

Unique values in 'Price' before cleaning: [  145    37    28   199   549   149   250    90   270   290   170    59
    49    68   285    75   100   150   700   125   175    40    89    95
    99   499   120    79   110   180   143   230   350   135    85    60
    70    55    44   200   165   115    74    84   129    50   185    80
   190   140    45    65   225   600   109  1990    73   240    72   105
   155   160    42   132   117   295   280   159   107    69   239   220
   399   130   375   585   275   139   260    35   133   300   289   179
    98   195    29    27    39   249   192   142   169  1000   131   138
   113   122   329   101   475   238   272   308   126   235   315   248
   128    56   207   450   215   210   385   445   136   247   118    77
    76    92   198   205   299   222   245   104   153   349   114   320
   292   226   420   500   325   307    78   265   108   123   189    32
    58    86   219   800   335    63   229   425    67    87  1200   158
   650   

In [26]:
police = pd.read_csv('mn_police_use_of_force.csv')
police.head()

,response_datetime,problem,is_911_call,primary_offense,subject_injury,force_type,force_type_action,race,sex,age,type_resistance,precinct,neighborhood
0,2016/01/01 00:47:36,Assault in Progress,Yes,DASLT1,NaN,Bodily Force,Body Weight to Pin,Black,Male,20.0,Tensed,1,Downtown East
1,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,Black,Female,27.0,Verbal Non-Compliance,1,Downtown West
2,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,White,Female,23.0,Verbal Non-Compliance,1,Downtown West
3,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West
4,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West


In [32]:
# Display unique values and value counts in the 'subject_injury' column before cleaning.
print("Unique values in 'subject_injury' before cleaning:", police['subject_injury'].unique(),'\n')
print("Value counts in 'subject_injury' (including NaN) before cleaning:\n", police['subject_injury'].value_counts(dropna=False), '\n')

# Replace missing values with 'Unknown'.
police_new= police.dropna(subset=['subject_injury'])

# Display unique values and value counts in the 'subject_injury' column after cleaning.
print("Unique values in 'subject_injury' after cleaning:", police_new['subject_injury'].unique(),'\n')
print("Value counts in 'subject_injury' (excluding NaN) after cleaning:\n", police_new['subject_injury'].value_counts(dropna=False), '\n')

# Cross-tabulate the cleaned 'subject_injury' variable with 'force_type'.
cross_tab = pd.crosstab(police['subject_injury_cleaned'], police['force_type'], margins=True, margins_name='Total')
print("Cross-tabulation of cleaned 'subject_injury' and 'force_type':\n", cross_tab)

Unique values in original 'subject_injury': [nan 'No' 'Yes'] 

Value counts in original 'subject_injury' (including NaN):
 subject_injury
NaN    9848
Yes    1631
No     1446
Name: count, dtype: int64 

Unique values in original 'subject_injury': ['No' 'Yes'] 

Value counts in original 'subject_injury' (excluding NaN):
 subject_injury
Yes    1631
No     1446
Name: count, dtype: int64 

Cross-tabulation of cleaned 'subject_injury' and 'force_type':
 force_type              Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury_cleaned                                                    
No                          0          1093                131        2   
Unknown                     2          7051               1421        0   
Yes                         2          1286                 41        0   
Total                       4          9430               1593        2   

force_type              Gun Point Display  Improvised Weapon  Less Lethal  \
subject_injury_clean

In [33]:
justice = pd.read_parquet('justice_data.parquet')
justice.head()

,InternalStudyID,REQ_REC#,Defendant_Sex,Defendant_Race,Defendant_BirthYear,Defendant_Age,Defendant_AgeGroup,Defendant_AgeatCurrentArrest,Defendant_AttorneyTypeAtCaseClosure,Defendant_IndigencyStatus,...,NewFelonySexualAssaultArrest_OffDate,NewFelonySexualAssaultArrest_ArrestDate,NewFelonySexualAssaultArrest_DaysBetweenContactEventandOffDate,NewFelonySexualAssaultArrest_DaysBetweenOffDateandArrestDate,NewFelonySexualAssaultArrest_DaysBetweenReleaseDateandOffDate,NewFelonySexualAssaultArrest_Disposition,Intertnalindicator_ReasonforExcludingFromFollowUpAnalysis,CriminalHistoryRecordsReturnedorCMSRecordsFoundforIndividual,DispRecordFoundforChargesinOct2017Contact_Atleast1dispfound,CrimeCommission2021ReportClassificationofDefendants
0,ADI00001,1,M,W,1986,31,3,31,99,99,...,,,,999,999,,4,1,0,Defendant could not be classified or tracked d...
1,ADI00007,3,M,B,1956,60,6,60,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
2,ADI00008,4,M,W,1990,27,3,27,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
3,CDI00036,6,M,B,1989,27,3,27,0,0,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
4,CDI00038,7,F,W,1988,28,3,28,0,0,...,,,,999,999,,0,1,1,New criminal offense punishable by incarcerati...


In [35]:
# Display unique values in the 'WhetherDefendantWasReleasedPretrial' column before cleaning.
print("Unique values in 'WhetherDefendantWasReleasedPretrial' before cleaning:", justice['WhetherDefendantWasReleasedPretrial'].unique(), '\n')

# Replace specified missing values with np.nan.
justice['WhetherDefendantWasReleasedPretrial'] = justice['WhetherDefendantWasReleasedPretrial'].replace([99, ''], np.nan)

print("Unique values in 'WhetherDefendantWasReleasedPretrial' after cleaning:", justice['WhetherDefendantWasReleasedPretrial'].unique(), '\n')
print("Number of missing values in 'WhetherDefendantWasReleasedPretrial' after cleaning:", justice['WhetherDefendantWasReleasedPretrial'].isnull().sum(), '\n')

# Display unique values in the 'ImposedSentenceAllChargeInContactEvent' column before cleaning and 'SentenceTypeAllChargesAtConvictionInContactEvent' column.
print("Unique values in 'ImposedSentenceAllChargeInContactEvent' before cleaning:", justice['ImposedSentenceAllChargeInContactEvent'].unique(), '\n')
print("Unique values in 'SentenceTypeAllChargesAtConvictionInContactEvent':", justice['SentenceTypeAllChargesAtConvictionInContactEvent'].unique(), '\n')

# Replace values in 'ImposedSentenceAllChargeInContactEvent' that represent missing/not applicable based on the 'SentenceTypeAllChargesAtConvictionInContactEvent' variable.
justice['ImposedSentenceAllChargeInContactEvent'] = justice['ImposedSentenceAllChargeInContactEvent'].replace([99999, ''], np.nan)

# Display unique and missing values the 'subject_injury' column after cleaning.
print("Unique values in 'ImposedSentenceAllChargeInContactEvent' after cleaning:", justice['ImposedSentenceAllChargeInContactEvent'].unique(), '\n')
print("Number of missing values in 'ImposedSentenceAllChargeInContactEvent' after cleaning:", justice['ImposedSentenceAllChargeInContactEvent'].isnull().sum(), '\n')

Unique values in 'WhetherDefendantWasReleasedPretrial' before cleaning: [9 0 1] 

Unique values in 'WhetherDefendantWasReleasedPretrial' after cleaning: [9 0 1] 

Number of missing values in 'WhetherDefendantWasReleasedPretrial' after cleaning: 0 

Unique values in 'ImposedSentenceAllChargeInContactEvent' before cleaning: [' ' '60' '12' '.985626283367556' '36' '6' '24' '5.91375770020534' '120'
 '72' '11.9917864476386' '0' '2.95687885010267' '84' '108' '300' '240'
 '180' '4' '96' '2' '54' '.328542094455852' '44' '5' '115' '132' '48'
 '258' '34' '76' '.164271047227926' '.131416837782341' '111' '9' '3'
 '1.97125256673511' '36.9856262833676' '.0657084188911704'
 '35.4928131416838' '106.492813141684' '8' '35' '18.3141683778234' '480'
 '32' '93' '234' '732' '1.16427104722793' '4.6570841889117' '21' '7'
 '4.49281314168378' '18' '600' '43.1642710472279' '179' '52' '30' '20'
 '192' '702' '14' '55' '53' '11.9055441478439' '114' '35.0061601642711'
 '68' '.657084188911704' '46.6242299794661' '102'

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

In [36]:
shark = pd.read_excel('GSAF5.xls')
shark.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,...,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,...,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,...,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,...,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,...,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
